In [ ]:
import pandas as pd
merged = pd.read_csv('Merged_fem_groups_with_user_chars_city_names.csv',  encoding='utf-8')
merged = merged.drop(columns=['80728963', '153276634', '189672959'])
merged.columns

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = 'notebook'

import vk_api
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px

# Read the access token from the file
with open('/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/TOKEN VK') as f:
    access_token = f.read().strip()

# Initialize VK session
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

# Ensure 'merged' DataFrame exists with the provided structure

# 1. Age Distribution
"""
merged['age'] = pd.to_datetime('today').year - pd.to_datetime(merged['bdate'], dayfirst=True, errors='coerce').dt.year
fig1 = px.histogram(merged, x='age', nbins=30, title='Age Distribution of Users',
                    labels={'age': 'Age'}, template='plotly_dark', marginal='box')
fig1.update_layout(xaxis_title='Age', yaxis_title='Count')
fig1.show()"""
# Count users without specified birthdate
missing_bdate_count = merged['bdate'].isna().sum()
specified_bdate_count = merged['bdate'].notna().sum()

# Calculate age as before
merged['age'] = pd.to_datetime('today').year - pd.to_datetime(merged['bdate'], dayfirst=True, errors='coerce').dt.year

# Create the histogram
fig1 = px.histogram(merged, x='age', nbins=30, title='Age Distribution of Users',
                    labels={'age': 'Age'}, template='plotly_dark', marginal='box')

# Add annotation for missing birthdate count
fig1.add_annotation(
    text=f"Percentage of users without specified birthdate: {round(missing_bdate_count/(specified_bdate_count + missing_bdate_count)*100, 2)}%",
    xref="paper", yref="paper",
    x=1, y=1.2, showarrow=False,
    font=dict(size=14, color="white"),
    bgcolor="rgba(0,0,0,0.7)",
    #bordercolor="white"
)

fig1.update_layout(xaxis_title='Age', yaxis_title='Count')
fig1.show()


# 2. Gender Ratio
gender_labels = {0: "Not Specified", 1: "Female", 2: "Male"}
merged['gender'] = merged['sex'].map(gender_labels)
fig2 = px.pie(merged, names='gender', title='Gender Distribution',
              color_discrete_sequence=px.colors.qualitative.Pastel)
fig2.update_traces(textinfo='percent+label')
fig2.show()

# 3. Top Cities
'''
city_counts = merged['city.name'].value_counts().reset_index()
city_counts.columns = ['city.name', 'count']
fig3 = px.bar(city_counts.head(10), x='city.name', y='count', title='Top 10 Cities by User Count',
              labels={'city.name': 'City', 'count': 'Number of Users'}, template='plotly')
fig3.update_layout(xaxis_title='City', yaxis_title='User Count', coloraxis_showscale=False)
fig3.show()
'''


# Count users who specified and didn't specify their city
unspecified_city_count = merged['city.name'].isna().sum()
specified_city_count = merged['city.name'].notna().sum()

# Create bar chart for top 10 cities
city_counts = merged['city.name'].value_counts().reset_index()
city_counts.columns = ['city.name', 'count']
fig3 = px.bar(city_counts.head(10), x='city.name', y='count', title='Top 10 Cities by User Count',
              labels={'city.name': 'City', 'count': 'Number of Users'}, template='plotly')
fig3.update_layout(xaxis_title='City', yaxis_title='User Count', coloraxis_showscale=False)

# Add annotation for unspecified city count
fig3.add_annotation(
    text=f"<b>Proportion of users without a specified city:</b> {round(unspecified_city_count / (unspecified_city_count + specified_city_count) * 100, 2)} %",
    xref="paper", yref="paper",
    x=1, y=1.2, showarrow=False,
    align="left",
    #bordercolor="black",
    #borderwidth=1,
    #bgcolor="lightyellow",
    #font=dict(size=12, color="black")
)

# Show the bar chart
fig3.show()

# Create a pie chart for proportions
labels = ['Specified City', 'Unspecified City']
values = [specified_city_count, unspecified_city_count]

fig_pie = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent', hole=0.4)])
fig_pie.update_layout(title='Proportion of Users Specifying a City', template='plotly')
fig_pie.show()


"""
# 7. Political Views
political_labels = {
    1: "Communist", 2: "Socialist", 3: "Moderate", 4: "Liberal",
    5: "Conservative", 6: "Monarchist", 7: "Ultra-Conservative",
    8: "Indifferent", 9: "Libertarian"
}
merged['political_view'] = merged['personal.political'].map(political_labels)
#fig4 = px.bar(merged['political_view'].value_counts().reset_index(), x='index', y='political_view',
#              title='Political View Distribution', labels={'index': 'Political View', 'political_view': 'Count'},
#              template='seaborn', color='index')
#fig4.update_layout(xaxis_title='Political View', yaxis_title='Count')
#fig4.show()

# Fix for Political Views Plot
political_counts = merged['political_view'].value_counts().reset_index()
political_counts.columns = ['Political View', 'Count']  # Rename columns explicitly

fig4 = px.bar(
    political_counts,
    x='Political View',
    y='Count',
    title='Political View Distribution',
    labels={'Political View': 'Political View', 'Count': 'Count'},
    template='seaborn',
    color='Political View'
)
fig4.update_layout(xaxis_title='Political View', yaxis_title='Count')
fig4.show()
"""

# Map political views
political_labels = {
    1: "Communist", 2: "Socialist", 3: "Moderate", 4: "Liberal",
    5: "Conservative", 6: "Monarchist", 7: "Ultra-Conservative",
    8: "Indifferent", 9: "Libertarian"
}
merged['political_view'] = merged['personal.political'].map(political_labels)

# Calculate political views distribution
political_counts = merged['political_view'].value_counts().reset_index()
political_counts.columns = ['Political View', 'Count']

# Count missing values
missing_political_views = merged['personal.political'].isna().sum()
specified_political_views = merged['personal.political'].notna().sum()

# Create the bar plot
fig4 = px.bar(
    political_counts,
    x='Political View',
    y='Count',
    title='Political View Distribution',
    labels={'Political View': 'Political View', 'Count': 'Count'},
    template='seaborn',
    color='Political View'
)

# Add annotation for missing values
fig4.add_annotation(
    text=f"Percentage of users <b>with</b> specified political views: {round(specified_political_views/(specified_political_views + missing_political_views) * 100, 2)} %",
    xref="paper", yref="paper",  # Relative to the entire figure
    x=0.9, y=1.1,  # Position the annotation above the chart
    showarrow=False,
    font=dict(size=12, color="black")
)

# Update layout and show plot
fig4.update_layout(xaxis_title='Political View', yaxis_title='Count')
fig4.show()

#=============================================================================
# 11. Popular Groups
#group_columns = [col for col in merged.columns if col.isdigit()]
#group_subscriptions = merged[group_columns].sum().sort_values(ascending=False).reset_index()
#group_subscriptions.columns = ['Group ID', 'Count']
#fig5 = px.bar(group_subscriptions.head(10), x='Group ID', y='Count', title='Top 10 Most Popular Groups',
#              labels={'Group ID': 'Group ID', 'Count': 'Number of Subscriptions'}, template='plotly')
#fig5.update_layout(xaxis_title='Group ID', yaxis_title='Subscription Count')
#fig5.show()




# Extract group columns (IDs)
group_columns = [col for col in merged.columns if col.isdigit()]

# Fetch group names using VK API
group_ids = group_columns
group_info = vk.groups.getById(group_ids=group_ids)

# Map group IDs to names
group_name_map = {str(group['id']): group['name'] for group in group_info}

# Create a new DataFrame with the group subscriptions
group_subscriptions = merged[group_columns].sum().sort_values(ascending=False).reset_index()
group_subscriptions.columns = ['Group ID', 'Count']

# Replace group IDs with names
group_subscriptions['Group Name'] = group_subscriptions['Group ID'].astype(str).map(group_name_map)
group_subscriptions = group_subscriptions.dropna(subset=['Group Name'])  # Drop any missing names if necessary

# Plot the top 10 most popular groups with names
fig5 = px.bar(group_subscriptions.head(10), x='Group Name', y='Count', title='Top 10 Most Popular Groups',
              labels={'Group Name': 'Group Name', 'Count': 'Number of Subscriptions'}, template='plotly')
fig5.update_layout(xaxis_title='Group Name', yaxis_title='Subscription Count')

# Show the plot
fig5.show()

#================================================================================
# 12. Correlations Between Groups
correlation_matrix = merged[group_columns].corr()
fig6 = px.imshow(correlation_matrix, title='Correlation Between Group Subscriptions',
                 labels=dict(x="Group ID", y="Group ID", color="Correlation"), color_continuous_scale='Viridis')
fig6.update_layout(template='plotly_dark')
fig6.show()


# Corr 2





# Extract group columns (IDs)
group_columns = [col for col in merged.columns if col.isdigit()]

# Fetch group names using VK API
group_ids = group_columns
group_info = vk.groups.getById(group_ids=group_ids)

# Map group IDs to names
group_name_map = {str(group['id']): group['name'] for group in group_info}

# Calculate correlation matrix and filter NaN values
correlation_matrix = merged[group_columns].corr().fillna(0)

# Map group IDs to names for rows and columns
correlation_matrix.index = correlation_matrix.index.map(lambda x: group_name_map.get(str(x), f"Group {x}"))
correlation_matrix.columns = correlation_matrix.columns.map(lambda x: group_name_map.get(str(x), f"Group {x}"))

# Create a heatmap to visualize correlations
fig65 = px.imshow(
    correlation_matrix,
    title='Correlation Between Group Subscriptions',
    labels=dict(x="Group Name", y="Group Name", color="Correlation"),
    color_continuous_scale='Viridis'
)

fig65.update_layout(template='plotly_dark')

# Show the plot
fig65.show()




# 13. Group Subscriptions Split by Sex and City
#grouped_by_sex_city = merged.groupby(['gender', 'city.name'])[group_columns].sum().stack().reset_index()
#grouped_by_sex_city.columns = ['Gender', 'City', 'Group ID', 'Count']
#fig7 = px.bar(grouped_by_sex_city, x='Group ID', y='Count', color='Gender',
#              facet_col='City', facet_col_wrap=3, title='Group Subscriptions by Gender and City',
#              labels={'Group ID': 'Group ID', 'Count': 'Number of Subscriptions'}, template='ggplot2')
#fig7.update_layout(xaxis_title='Group ID', yaxis_title='Subscription Count')
#fig7.show()

# Get the top 10 cities by user count
top_cities = merged['city.name'].value_counts().head(10).index

# Filter the dataset to only include users from these cities
filtered_data = merged[merged['city.name'].isin(top_cities)]

# Group and aggregate subscription counts by gender and city
grouped_data = filtered_data.groupby(['gender', 'city.name'])[group_columns].sum().stack().reset_index()
grouped_data.columns = ['Gender', 'City', 'Group ID', 'Count']

# Create the bar chart
fig7 = px.bar(grouped_data, x='Group ID', y='Count', color='Gender',
              facet_col='City', title='Group Subscriptions by Gender and Top Cities',
              labels={'Group ID': 'Group ID', 'Count': 'Number of Subscriptions'},
              template='ggplot2', facet_col_wrap=3)

fig7.update_layout(xaxis_title='Group ID', yaxis_title='Subscription Count')
fig7.show()
